# Imports

First we have to import some packages to use down the line

In [ ]:
import sys
import xgboost as xgb
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.plotting import add_at_risk_counts
import sys
from tableone import TableOne
import sklearn
import math
import statsmodels.api as sm
import warnings
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, LassoLarsIC, LassoCV, LassoLarsCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.calibration import calibration_curve
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import log_loss
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import imblearn
import scipy
import re
from imblearn.over_sampling import SMOTE
import time
import datetime
import openpyxl
import numpy.random as rng
import seaborn as sns
from scipy import stats
import pickle

from ncdb_tools import *

# Loading the NCDB File

First step is to properly load the NCDB file.

Since loading the full NCDB csv takes quite some time, you can utilize the savefile and loadfile specifications to save a subset of the database once you've narrowed down to the patients / variables you are interested in.

The import function's name is load_data_her2. After the function, we specify parameters (filename, savefile, loadfile, and lower). You can provide default values for these parameters if you want using the equals sign - this means that the user doesn't need to provide these parameters every time the function is called.

Detailed descriptions of all NCDB elements are found here:
<a href="https://www.facs.org/-/media/files/quality-programs/cancer/ncdb/puf_data_dictionary_2017.ashx">https://www.facs.org/-/media/files/quality-programs/cancer/ncdb/puf_data_dictionary_2017.ashx</a>

https://www.facs.org/media/4tzpgfsb/ncdb-puf-quickstart-data-structure-2020.pdf

'Site Specific Factors' are found here:
<a href="https://web2.facs.org/cstage0205/breast/Breastschema.html">https://web2.facs.org/cstage0205/breast/Breastschema.html</a>

Let's demonstrate the use of our above NCDB accessing function. Replace the filename with your copy of NCDB.

This code will take some time to execute! Try not to rerun it

In [ ]:
df = load_data_her2(
    filename = r"/mnt/data/NCDB/NCDBPUF_Breast.0.2020.csv",
    lower = True
)

In [ ]:
#Adds the relevant important columns to the dataframe
df_new = getNCDBClassifications(df, use_imputation = False)
df_new = addTennNomogram(df_new)

# Minimal Feature Set
How can we logically exclude missing variables? Let's choose variables from the overall dataset that are 1) most predictive and 2) least missing

In [ ]:
df_new = transformVar(df_new, 'age')
df_new = transformVar(df_new, 'er_num')
df_new = transformVar(df_new, 'pr_num')
df_new = transformVar(df_new, 'ki67_num')
df_new = transformVar(df_new, 'tumor_size')
print(df_new.columns)

maxList = ['age','sex', 'grade_med', 'grade_high', 'tumor_size', 'er', 'pr', 'regional_nodes_positive', 'chest_wall', 'skin_changes', 'inflammatory', 'lvi', 
                     'ductal', 'lobular', 'ductlob', 'mucinous', 'papillary', 'tubular', 'medullary', 'metaplastic', 'paget', 'sarcoma',
                     'asian', 'black', 'hispanic', 'native', 'her2', 'er_num', 'pr_num', 'ki67_num', 'her2copies', 'her2ratio']
#maxList += ['age_' + x for x in ['log', 'sq', 'sqrt', 'cbrt']]
#maxList += ['er_num_' + x for x in ['log', 'sq', 'sqrt', 'cbrt']]
#maxList += ['pr_num_' + x for x in ['log', 'sq', 'sqrt', 'cbrt']]
#maxList += ['ki67_num_' + x for x in ['log', 'sq', 'sqrt', 'cbrt']]
#maxList += ['tumor_size_' + x for x in ['log', 'sq', 'sqrt', 'cbrt']]

mlf_trans = ['age_log','sex', 'grade_med', 'grade_high', 'tumor_size_log', 'er', 'pr', 'regional_nodes_positive', 'chest_wall', 'skin_changes', 'inflammatory', 'lvi', 
                     'ductal', 'lobular', 'ductlob', 'mucinous', 'papillary', 'tubular', 'medullary', 'metaplastic', 'paget', 'sarcoma',
                     'asian', 'black', 'hispanic', 'native', 'her2', 'er_num_sq', 'pr_num', 'ki67_num_sqrt', 'her2copies', 'her2ratio']

res = selectBaseInterestingFeatures(df_new, maxList)

In [ ]:
res_sorted = dict(sorted(res.items(), key=lambda x:x[1]['llr'], reverse = True))
n = len(res)
width = 0.25
  
fig, ax1 = plt.subplots(dpi = 300)
l1 = ax1.bar(np.arange(n), [x[1]['llr'] for x in res_sorted.items()], color = 'b',
        width = width, edgecolor = 'black',
        )
ax1.set_xticks(np.arange(n) + width/2)
ax1.set_xticklabels([x[0] for x in res_sorted.items()], rotation = 90, ha='center', size = 6)

ax2 = ax1.twinx()
l2 = ax2.bar(np.arange(n) + width, [x[1]['sample'] for x in res_sorted.items()], color = 'g',
        width = width, edgecolor = 'black',
        )

plt.legend([l1, l2], ["LLR", "N"])
plt.xlabel("Feature")
ax1.set_ylabel("LLR")
ax2.set_ylabel("Sample Size")

plt.tight_layout()
  
plt.show()

In [ ]:
#Minimal Feature Set:
mfs = ['age','sex', 'grade_med', 'grade_high', 'tumor_size', 'er', 'pr', 'regional_nodes_positive', 'skin_changes', 'lvi', 'chest_wall',
                     'ductal', 'lobular', 'ductlob', 'mucinous', 'tubular', 'medullary', 'metaplastic', 'paget', 'sarcoma', 'papillary', 'inflammatory',
                     'hispanic', 'native', 'asian', 'black', 'her2', 'er_num', 'pr_num', 'ki67_num']

#MFS with transforms
mfs = ['age', 'age_log', 'age_sq', 'age_sqrt', 'age_cbrt', 'sex', 'grade_med', 'grade_high', 
                             'tumor_size', 'tumor_size_log', 'tumor_size_sq', 'tumor_size_sqrt', 'tumor_size_cbrt', 'er', 'pr', 
                             'regional_nodes_positive', 'skin_changes', 'lvi', 'chest_wall', 'ductal', 'lobular', 'ductlob', 'mucinous', 'tubular', 
                             'medullary', 'metaplastic', 'paget', 'sarcoma', 'papillary', 'inflammatory', 'hispanic', 'native', 'asian', 'black', 'her2', 'er_num_sq', 'pr_num', 'ki67_num_sqrt']


# Model Training

In [ ]:
print(len(df_test_ncdb.alive))

In [ ]:
print("Excluding stage 0: " + str(len(df_new.index)))

df_train = df_new[(df_new.regional_nodes_positive < 4)]
print("Excluding 4 or more nodes positive: " + str(len(df_train.index)))

df_train = df_train.dropna(subset = ['odx', 'high_odx'])
print("Excluding patients missing ODX: " + str(len(df_train.index)))

df_train = df_train.dropna(subset = mfs)
print("Excluding patients missing minimal feature set: " + str(len(df_train.index)))

#Add life expectancies
df_train['life_expectancy'] = df_train.apply(lambda row: femaleLife[int(row['age'])] if row['sex'] == 0 else maleLife[int(row['age'])] if row['sex'] == 1 else None, axis=1)

#Split into training and test sets
df_train, df_test_ncdb = sklearn.model_selection.train_test_split(
    df_train,
    test_size = 0.2,
    random_state = 1
)

In [ ]:
model = LogisticRegression(max_iter = 1000, n_jobs = -1)
params = None

In [ ]:
%run ncdb_tools.py

In [ ]:
best_model_features = make_best_model_features()

for model_name in model_names:
    best_model_features[model_name]['results'] = compareClassifiersNCDB(
        df_train,
        model = model,
        params = params,
        maximizeAIC = "train",
        includeTransforms = "all",
        doWarmStart = False,
        **best_model_features[model_name]['arguments']
    )

In [ ]:
best_model_features = rename_models(best_model_features)

# ROC Analyses

In [ ]:
for model_name in model_names + ['Tennessee Nomogram']:
    best_model_features[model_name]['subgroup_analyses'] = {
        'histologic': None,
        'race': None,
        'nodes': {
            'Negative': {},
            'Positive': {}
        },
        'training': {
            'all': {},
            'test': {}
        }
    }
    
    best_model_features[model_name]['subgroup_analyses']['histologic'] = {k:{} for k in histologic_dict.keys()}
    best_model_features[model_name]['subgroup_analyses']['race'] = {k:{} for k in race_dict.keys()}

## NCDB Model Assessment
Using a restricted dataset that contains all the features of interest across all compared models, compare the AUC in the NCDB cohort.

In [ ]:
# Use all of training data to determine sens/spec thresholds

fig, ax = plt.subplots(dpi = 300)

for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results'] = plotAUROCFeat(
        ax,
        df_train,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        average = True
    )
    
plt.close()

In [ ]:
# Use only test subset to determine AUROCs and plot

fig, ax = plt.subplots(dpi = 300)
    
for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['training']['test']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train, df_test_ncdb,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.legend()
plt.show()

## UCMC Model Validation

In [ ]:
df_ucmc = pd.read_csv(r"/mnt/data/UCMC/RS_dataset.csv", dtype = {'nodegrp': str})

In [ ]:
df_ucmc_parse = getUCMCClassifications(df_ucmc, use_imputation = False)
#df_ucmc_parse = addTennNomogram_UCMC(df_ucmc_parse, imputeMissing = False)
df_ucmc_parse = transformVar(df_ucmc_parse, 'age')
df_ucmc_parse = transformVar(df_ucmc_parse, 'er_num')
df_ucmc_parse = transformVar(df_ucmc_parse, 'pr_num')
df_ucmc_parse = transformVar(df_ucmc_parse, 'ki67_num')
df_ucmc_parse = transformVar(df_ucmc_parse, 'tumor_size')

In [ ]:
#only including patients with < 4 lymph nodes positive and at least ER or PR positive
df_ucmc_parse = df_ucmc_parse.loc[
    (df_ucmc_parse.regional_nodes_positive < 4) &
    ((df_ucmc_parse.er == 1) | (df_ucmc_parse.pr == 1))
]

#drop patients with missing data
for model_name in model_names:
    df_ucmc_parse = df_ucmc_parse.dropna(subset = best_model_features[model_name]['features'])

In [ ]:
df_ucmc_parse_odx_val = df_ucmc_parse.dropna(subset = ['high_odx'])
for model_name in model_names:
    df_ucmc_parse_odx_val = df_ucmc_parse_odx_val.dropna(subset = best_model_features[model_name]['features'])

In [ ]:
df_ucmc_parse_survival_val = df_ucmc_parse.copy()
for model_name in model_names:
    df_ucmc_parse_survival_val = df_ucmc_parse_survival_val.dropna(subset = best_model_features[model_name]['features'])

In [ ]:
#evaluate models
fig, ax = plt.subplots(dpi = 300)

for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external']['senspec_results'] = plotAUROCFeatExternal(
        ax,
        df_train,
        df_ucmc_parse_odx_val,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.legend()
plt.show()

Model assessment comparison, including DL Path predictions.

In [ ]:
df_ucmc_parse_odx_val_dl = df_ucmc_parse.dropna(subset = ['high_odx', 'percent_tiles_positive0'])
for model_name in model_names:
    df_ucmc_parse_odx_val_dl = df_ucmc_parse_odx_val_dl.dropna(subset = best_model_features[model_name]['features'])

#evaluate models
fig, ax = plt.subplots(dpi = 300)

for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external']['senspec_results'] = plotAUROCFeatExternal(
        ax,
        df_train,
        df_ucmc_parse_odx_val_dl,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

best_model_features['DL Path'] = {}
best_model_features['DL Path']['subgroup_analyses'] = {}
best_model_features['DL Path']['subgroup_analyses']['external'] = {}
best_model_features['DL Path']['subgroup_analyses']['external']['external'] = {}
best_model_features['DL Path']['subgroup_analyses']['external']['external']['senspec_results'] = plotAUROCFeatExternal(
     ax,
     df_train,
     df_ucmc_parse_odx_val_dl,
     ['comb'],
     label = "DL Path",
     useColumn = 'comb'
)


ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.legend()
plt.show()

Using DeLong's method, compute z-scores and p-values for difference in AUC of models in the validation cohort.

In [ ]:
auc_compare_z = pd.DataFrame(columns = model_names, index = model_names)
auc_compare_pval = pd.DataFrame(columns = model_names, index = model_names)

for m1 in model_names:
    for m2 in model_names:
        auc_compare_z.loc[m1, m2], auc_compare_pval.loc[m1, m2] = AUROCFeatExternal_zp(
            df_train,
            df_ucmc_parse_odx_val,
            feat = best_model_features[m1]['features'],
            feat_comparison = best_model_features[m2]['features'],
            model = model
        )
        
auc_compare_pval = 10**auc_compare_pval #initially returned as log10(pval)

In [ ]:

auc_compare_pval

In [ ]:
auc_compare_z = pd.DataFrame(columns = model_names, index = model_names)
auc_compare_pval = pd.DataFrame(columns = model_names, index = model_names)

for m1 in model_names:
    for m2 in model_names:
        auc_compare_z.loc[m1, m2], auc_compare_pval.loc[m1, m2] = AUROCFeatExternal_zp(
            df_train,
            df_test_ncdb,
            feat = best_model_features[m1]['features'],
            feat_comparison = best_model_features[m2]['features'],
            model = model
        )
        
auc_compare_pval_test = 10**auc_compare_pval #initially returned as log10(pval)

In [ ]:

auc_compare_pval_test

## Race Subgroup Analyses

In [ ]:
# Individual Figures
for model_name in model_name_legend_label_dict.keys():
    fig, ax = plt.subplots(dpi = 300)
    
    for race_name, col_id in race_dict.items():    
        print(race_name, model_name)
        best_model_features[model_name]['subgroup_analyses']['race'][race_name]['senspec_results'] = plotAUROCFeat_Test(
            ax,
            df_train,
            df_test_ncdb,
            best_model_features[model_name]['features'],
            label = race_name,
            plot_color = race_colors_dict[race_name],
            subgroup_type = 'race',
            subgroup_id = col_id,
            thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
        )
    
    ax.legend()
    plt.show()

In [ ]:
# 4 Panel Figure
fig, axs = plt.subplots(2, 2, dpi = 300)

for model_name, ax in zip(model_name_legend_label_dict.keys(), axs.ravel()):
    for race_name, col_id in race_dict.items():
        print(race_name, model_name)
        best_model_features[model_name]['subgroup_analyses']['race'][race_name]['senspec_results'] = plotAUROCFeat_Test(
            ax,
            df_train,
            df_test_ncdb,
            best_model_features[model_name]['features'],
            label = race_name,
            auc_in_legend = False,
            plot_color = race_colors_dict[race_name],
            subgroup_type = 'race',
            subgroup_id = col_id,
            thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
        )
        
    ax.title.set_text(model_name)
        
handles, labels = ax.get_legend_handles_labels()
lgd = fig.legend(handles, labels, loc='right', bbox_to_anchor=(1.35,0.5))

plt.tight_layout()
plt.show()

## Histologic Subgroup Analyses

In [ ]:
min_sample_size = 50 # enforcing minimum sample size for testing subgroup performance

In [ ]:
fig, ax = plt.subplots(dpi = 300)

for histo_name, col_id in histologic_dict.items():
    df_subgroup = df_train[
        df_train.histology == col_id
    ]
    
    print(histo_name, "Tennessee Nomogram")
    
    if (df_subgroup.shape[0] > min_sample_size) & (df_subgroup['high_odx'].sum() > 1):
        best_model_features['Tennessee Nomogram']['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = plotAUROCFeat_Test(
            ax,
            df_train,
            df_test_ncdb,
            ['ten_score'],
            label = histo_name,
            useColumn = 'ten_score',
            plot_color = histologic_colors_dict[histo_name],
            subgroup_type = 'histologic',
            subgroup_id = col_id,
            thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
        )
    else:
        best_model_features['Tennessee Nomogram']['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = None


ax.legend()
plt.show()

In [ ]:
for model_name in model_name_legend_label_dict.keys():
    fig, ax = plt.subplots(dpi = 300)
    
    for histo_name, col_id in histologic_dict.items():
        df_subgroup = df_train[
            df_train.histology == col_id
        ]
    
        print(histo_name, model_name)
        
        if (df_subgroup.shape[0] > min_sample_size) & (df_subgroup['high_odx'].sum() > 1):
            best_model_features[model_name]['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = plotAUROCFeat_Test(
                ax,
                df_train,
                df_test_ncdb,
                best_model_features[model_name]['features'],
                label = histo_name,
                plot_color = histologic_colors_dict[histo_name],
                subgroup_type = 'histologic',
                subgroup_id = col_id,
                thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
            )
        else:
            best_model_features[model_name]['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = None
            
    ax.legend()
    plt.show()

In [ ]:
# 4 Panel Figure
fig, axs = plt.subplots(2, 2, dpi = 300)

for model_name, ax in zip(model_name_legend_label_dict.keys(), axs.ravel()):
    for histo_name, col_id in histologic_dict.items():  
        df_subgroup = df_train[
            df_train.histology == col_id
        ]
        
        print(histo_name, model_name)
        
        if (df_subgroup.shape[0] > min_sample_size) & (df_subgroup['high_odx'].sum() > 1):
            best_model_features[model_name]['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = plotAUROCFeat_Test(
                ax,
                df_train,
                df_test_ncdb,
                best_model_features[model_name]['features'],
                label = histo_name,
                auc_in_legend = False,
                plot_color = histologic_colors_dict[histo_name],
                subgroup_type = 'histologic',
                subgroup_id = col_id,
                thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
            )
        else:
            best_model_features[model_name]['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = None
            
    ax.title.set_text(model_name)
    
handles, labels = axs.ravel()[0].get_legend_handles_labels()
lgd = fig.legend(handles, labels, loc='right', bbox_to_anchor=(1.35,0.5))

plt.tight_layout()
plt.show()

## Node Subgroup Analyses

In [ ]:
fig, ax = plt.subplots(dpi = 300)

print("Negative", "Tennesse Nomogram")

best_model_features['Tennessee Nomogram']['subgroup_analyses']['nodes']['Negative']['senspec_results'] = plotAUROCFeat_Test(
    ax,
    df_train,
    df_test_ncdb,
    ['ten_score'],
    label = "Negative",
    useColumn = 'ten_score',
    subgroup_type = 'nodes',
    subgroup_id = 0,
    thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
)

print("Positive", "Tennessee Nomogram")

best_model_features['Tennessee Nomogram']['subgroup_analyses']['nodes']['Positive']['senspec_results'] = plotAUROCFeat_Test(
    ax,
    df_train,
    df_test_ncdb,
    ['ten_score'],
    label = "Positive",
    useColumn = 'ten_score',
    subgroup_type = 'nodes',
    subgroup_id = 1,
    thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
)

ax.legend()
plt.show()

In [ ]:
for model_name in model_name_legend_label_dict.keys():
    fig, ax = plt.subplots(dpi = 300)

    print("Negative", model_name)

    best_model_features[model_name]['subgroup_analyses']['nodes']['Negative']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        label = "Negative",
        subgroup_type = 'nodes',
        subgroup_id = 0,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )
    
    print("Positive", model_name)

    best_model_features[model_name]['subgroup_analyses']['nodes']['Positive']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        label = "Positive",
        subgroup_type = 'nodes',
        subgroup_id = 1,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )
    
    ax.legend()
    plt.show()

In [ ]:
# 4 Panel Figure
fig, axs = plt.subplots(2, 2, dpi = 300)

for model_name, ax in zip(model_name_legend_label_dict.keys(), axs.ravel()):
    # Negative
    print("Negative", model_name)

    best_model_features[model_name]['subgroup_analyses']['nodes']['Negative']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        auc_in_legend = False,
        label = "Negative",
        plot_color = nodes_colors_dict['Negative'],
        subgroup_type = 'nodes',
        subgroup_id = 0,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )
    
    # Postive Post-Menopausal
    print("Positive", model_name)
    
    best_model_features[model_name]['subgroup_analyses']['nodes']['Positive']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        auc_in_legend = False,
        label = "Positive",
        plot_color = nodes_colors_dict['Positive'],
        subgroup_type = 'nodes',
        subgroup_id = 1,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

    ax.title.set_text(model_name)
        
handles, labels = axs.ravel()[0].get_legend_handles_labels()
lgd = fig.legend(handles, labels, loc='right', bbox_to_anchor=(1.35,0.5))

plt.tight_layout()
plt.show()

In [ ]:
# 9 Panel Figure

fig, axs = plt.subplots(3, 3, dpi = 300, figsize = (10,8), sharex = True, sharey = True)

for model_name, ax in zip(model_name_legend_label_dict.keys(), axs.ravel(order = 'F')[:3]):
    for race_name, col_id in race_dict.items():
        print(race_name, model_name)
        best_model_features[model_name]['subgroup_analyses']['race'][race_name]['senspec_results'] = plotAUROCFeat_Test(
            ax,
            df_train,
            df_test_ncdb,
            best_model_features[model_name]['features'],
            label = race_name,
            auc_in_legend = False,
            plot_color = race_colors_dict[race_name],
            subgroup_type = 'race',
            subgroup_id = col_id,
            thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
        )
        
    ax.title.set_text(model_name_legend_label_dict[model_name])
        
handles, labels = ax.get_legend_handles_labels()
lgd = fig.legend(handles, labels, loc = 'upper left', bbox_to_anchor=(0.035,0))

for model_name, ax in zip(model_name_legend_label_dict.keys(), axs.ravel(order = 'F')[3:6]):
    for histo_name, col_id in histologic_dict.items():  
        df_subgroup = df_train[
            df_train.histology == col_id
        ]
        
        print(histo_name, model_name)
        
        if (df_subgroup.shape[0] > min_sample_size) & (df_subgroup['high_odx'].sum() > 1):
            best_model_features[model_name]['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = plotAUROCFeat_Test(
                ax,
                df_train,
                df_test_ncdb,
                best_model_features[model_name]['features'],
                label = histo_name,
                auc_in_legend = False,
                plot_color = histologic_colors_dict[histo_name],
                subgroup_type = 'histologic',
                subgroup_id = col_id,
                thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
            )
        else:
            best_model_features[model_name]['subgroup_analyses']['histologic'][histo_name]['senspec_results'] = None
            
    ax.title.set_text(model_name_legend_label_dict[model_name])
    
handles, labels = ax.get_legend_handles_labels()
lgd = fig.legend(handles, labels, loc = 'upper left', bbox_to_anchor=(0.36,0))

for model_name, ax in zip(model_name_legend_label_dict.keys(), axs.ravel(order = 'F')[6:]):
    # Negative
    print("Negative", model_name)

    best_model_features[model_name]['subgroup_analyses']['nodes']['Negative']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        auc_in_legend = False,
        label = "Negative",
        plot_color = nodes_colors_dict['Negative'],
        subgroup_type = 'nodes',
        subgroup_id = 0,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

    # Postive Post-Menopausal
    print("Positive", model_name)

    best_model_features[model_name]['subgroup_analyses']['nodes']['Positive']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        auc_in_legend = False,
        label = "Positive",
        plot_color = nodes_colors_dict['Positive'],
        subgroup_type = 'nodes',
        subgroup_id = 1,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

    ax.title.set_text(model_name_legend_label_dict[model_name])
        
handles, labels = ax.get_legend_handles_labels()
lgd = fig.legend(handles, labels, loc = 'upper left', bbox_to_anchor=(0.68,0))

plt.figtext(0.188, 1.0005, 'Race', fontweight = "bold", size = 12, ha = 'center')
plt.figtext(0.516, 1.0005, 'Histology', fontweight = "bold", size = 12, ha = 'center')
plt.figtext(0.825, 1.0005, 'Nodes', fontweight = "bold", size = 12, ha = 'center')

for ax, l in zip(axs.ravel(order = 'F'), ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']):
    ax.set_title(l, loc='left', fontweight='bold')

plt.tight_layout()
plt.show()

## Compiling Results

In [ ]:
subgroup_results_df = pd.DataFrame(
    columns = [
        'model',
        'subgroup_type',
        'subgroup_group',
        'n_patients',
        'features',
        'n_features',
        '95pSens_thres',
        '95pSens_sens',
        '95pSens_spec',
        '95pSpec_thres',
        '95pSpec_sens',
        '95pSpec_spec',
        'AUC',
        'AUC_lowerbound',
        'AUC_upperbound',
        'AUPRC',
        'AUPRC_lowerbound',
        'AUPRC_upperbound'
    ]
)

for model_name in model_names:
    for subgroup_type in best_model_features[model_name]['subgroup_analyses'].keys():
        for subgroup_group in best_model_features[model_name]['subgroup_analyses'][subgroup_type].keys():
            print(model_name, subgroup_type, subgroup_group)
            results = best_model_features[model_name]['subgroup_analyses'][subgroup_type][subgroup_group]['senspec_results']
            
            if results != None:
                subgroup_results_df = subgroup_results_df.append({
                    'model': model_name,
                    'subgroup_type': subgroup_type,
                    'subgroup_group': subgroup_group,
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    '95pSens_thres': results['95% Sensitivity']['Threshold'],
                    '95pSens_sens': results['95% Sensitivity']['Sensitivity'],
                    '95pSens_spec': results['95% Sensitivity']['Specificity'],
                    '95pSpec_thres': results['95% Specificity']['Threshold'],
                    '95pSpec_sens': results['95% Specificity']['Sensitivity'],
                    '95pSpec_spec': results['95% Specificity']['Specificity'],
                    'AUC': results['AUC']['AUC'],
                    'AUC_lowerbound': results['AUC']['Lower Bound'],
                    'AUC_upperbound': results['AUC']['Upper Bound'],
                    'AUPRC': results['AUPRC']['AUPRC'],
                    'AUPRC_lowerbound': results['AUPRC']['Lower Bound'],
                    'AUPRC_upperbound': results['AUPRC']['Upper Bound']
                }, ignore_index = True)
            else:
                subgroup_results_df = subgroup_results_df.append({
                    'model': model_name,
                    'subgroup_type': subgroup_type,
                    'subgroup_group': subgroup_group,
                    'n_patients': None,
                    'features': None,
                    'n_features': None,
                    '95pSens_thres': None,
                    '95pSens_sens': None,
                    '95pSens_spec': None,
                    '95pSpec_thres': None,
                    '95pSpec_sens': None,
                    '95pSpec_spec': None,
                    'AUC': None,
                    'AUC_lowerbound': None,
                    'AUC_upperbound': None,
                    'AUPRC': None,
                    'AUPRC_lowerbound': None,
                    'AUPRC_upperbound': None
                }, ignore_index = True)

In [ ]:
subgroup_results_df['auc_lb'] = subgroup_results_df['AUC'] - subgroup_results_df['AUC_lowerbound']
subgroup_results_df['auc_ub'] = subgroup_results_df['AUC_upperbound'] - subgroup_results_df['AUC']

In [ ]:
p = plot_errbar(
    data = subgroup_results_df.dropna(subset = ['AUC']),
    subgroup_type = 'race'
)

In [ ]:
subgroup_results_df[
    subgroup_results_df.subgroup_type == 'race'
].pivot(
    values = ['n_patients', 'AUC', 'AUC_lowerbound', 'AUC_upperbound', 'AUPRC', 'AUPRC_lowerbound', 'AUPRC_upperbound'],
    columns = ['subgroup_group'],
    index = ['model']
).swaplevel(0, 1, axis = 1).sort_index(axis = 1).loc[model_names].transpose().dropna()

In [ ]:
p = plot_errbar(
    data = subgroup_results_df.dropna(subset = ['AUC']),
    subgroup_type = 'histologic'
)

In [ ]:
subgroup_results_df[
    subgroup_results_df.subgroup_type == 'histologic'
].pivot(
    values = ['n_patients', 'AUC', 'AUC_lowerbound', 'AUC_upperbound', 'AUPRC', 'AUPRC_lowerbound', 'AUPRC_upperbound'],
    columns = ['subgroup_group'],
    index = ['model']
).swaplevel(0, 1, axis = 1).sort_index(axis = 1).loc[model_names].transpose().dropna()

In [ ]:
p = plot_errbar(
    data = subgroup_results_df.dropna(subset = ['AUC']),
    subgroup_type = 'nodes'
)

In [ ]:
subgroup_results_df[
    subgroup_results_df.subgroup_type == 'nodes'
].pivot(
    values = ['n_patients', 'AUC', 'AUC_lowerbound', 'AUC_upperbound', 'AUPRC', 'AUPRC_lowerbound', 'AUPRC_upperbound'],
    columns = ['subgroup_group'],
    index = ['model']
).swaplevel(0, 1, axis = 1).sort_index(axis = 1).loc[model_names].transpose()

## Generating Baseline Demographics

In [ ]:
baselineCharacteristics(pd.concat([df_train, df_test_ncdb]))

In [ ]:
baselineCharacteristics_external(df_ucmc_parse_survival_val.copy())

## Manuscript Figures

In [ ]:
fig, ax = plt.subplots(dpi = 300, figsize=(1.5, 1.5))
    
for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['training']['test']['senspec_results'] = plotAUROCFeat_Test(
        ax,
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines[['right', 'top']].set_visible(False)
plt.show()

In [ ]:
#evaluate models

fig, ax = plt.subplots(dpi = 300, figsize=(1.5, 1.5))
    
for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external']['senspec_results'] = plotAUROCFeatExternal(
        ax,
        df_train,
        df_ucmc_parse_odx_val,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines[['right', 'top']].set_visible(False)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, dpi = 300, figsize = (10.5, 4), sharey = True)

for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['training']['test']['senspec_results'] = plotAUROCFeat_Test(
        ax[0],
        df_train,
        df_test_ncdb,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

ax[0].set_title('A', loc='left', fontweight='bold')
ax[0].set_xlabel('1 - Specificity')
ax[0].set_ylabel('Sensitivity')
ax[0].legend()
    
for model_name in model_names:
    print(model_name)
    best_model_features[model_name]['subgroup_analyses']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external'] = {}
    best_model_features[model_name]['subgroup_analyses']['external']['external']['senspec_results'] = plotAUROCFeatExternal(
        ax[1],
        df_train,
        df_ucmc_parse_odx_val,
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        model = model,
        thresholds = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']
    )

ax[1].set_title('B', loc='left', fontweight='bold')
ax[1].set_xlabel('1 - Specificity')
ax[1].set_ylabel('Sensitivity')
ax[1].legend()

plt.tick_params('y', labelleft=True)

plt.tight_layout()
plt.show()

In [ ]:
survival_results_data = {k:{c:{'95% Sensitivity': None, '90% Sensitivity': None} for c in model_names} for k in ['training', 'validation']}

In [ ]:
##Validation Survival Curves in High/Low ODX at 90% Sensitivity Cutoff

cutoff_level = '90% Sensitivity'
cutoff = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results'][cutoff_level]['Threshold']

fig, axs = plt.subplots(3, 2, dpi = 300, figsize = (8,12), sharey = True, sharex = True)

i = 0

for h_odx in [0, 1]:
    for model_name in model_names:
        print(cutoff_level, model_name)
        ax = axs.ravel(order = 'F')[i]
        try:
            survival_results_data['validation'][model_name][cutoff_level] = plotSurvivalFeatExternal(
            ax,
            df_train,
            df_ucmc_parse_survival_val[(df_ucmc_parse_survival_val.regional_nodes_positive < 4) & (df_ucmc_parse_survival_val.high_odx == h_odx)],
            best_model_features[model_name]['features'],
            label = model_name_legend_label_dict[model_name],
            cutoff = cutoff,
            model = model
            )
        
            if survival_results_data['validation'][model_name][cutoff_level]['Univariate']['HR']['Upper Bound'] > 10000:
                ax.annotate("HR: N/A", (0, 0.61))
            else:
                ax.annotate(stringHR_from_data(survival_results_data['validation'][model_name][cutoff_level]['Univariate']), (0, 0.61))
        except:
            pass
        i = i + 1
        
for ax, l in zip(axs.ravel(order = 'F'), ['A', 'B', 'C', 'D', 'E', 'F']):
    try:
        ax.get_legend().remove()
    except:
        pass
    ax.set_title(l, loc='left', fontweight='bold')
    ax.set_ylim([0.60, None])
    ax.set_xlabel('Months')
    ax.set_ylabel('Recurrence Free Interval')

plt.figtext(0.25, 1.0005, 'Low ODX', fontweight = "bold", size = 12)
plt.figtext(0.75, 1.0005, 'High ODX', fontweight = "bold", size = 12)
axs[0,1].legend(['Low Risk', 'High Risk'], loc='upper right')
plt.tight_layout()
fig.show()

In [ ]:
##Validation Survival Curves

cutoff_levels = [
    '95% Sensitivity',
    '90% Sensitivity'
]

fig, axs = plt.subplots(3, 2, dpi = 300, figsize = (8,12), sharey = True, sharex = True)

i = 0

for cutoff_level in cutoff_levels:    
    for model_name in model_names:
        cutoff = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results'][cutoff_level]['Threshold']

        print(cutoff_level, cutoff, model_name)
        ax = axs.ravel(order = 'F')[i]
        
        survival_results_data['validation'][model_name][cutoff_level] = plotSurvivalFeatExternal(
        ax,
        df_train,
        df_ucmc_parse_survival_val[(df_ucmc_parse_survival_val.regional_nodes_positive < 4)],
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        cutoff = cutoff,
        model = model
        )
        i = i + 1
        
        if math.isinf(survival_results_data['validation'][model_name][cutoff_level]['Univariate']['HR']['Upper Bound']):
            ax.annotate("HR: N/A", (0, 0.905))
        else:
            ax.annotate(stringHR_from_data(survival_results_data['validation'][model_name][cutoff_level]['Univariate']), (0, 0.905))

for ax, l in zip(axs.ravel(order = 'F'), ['A', 'B', 'C', 'D', 'E', 'F']):
    ax.get_legend().remove()
    ax.set_title(l, loc='left', fontweight='bold')
    ax.set_ylim([0.90, None])
    ax.set_xlabel('Months')
    ax.set_ylabel('Recurrence Free Interval')

plt.figtext(0.22, 1.0005, '95% Sensitivity', fontweight = "bold", size = 12)
plt.figtext(0.71, 1.0005, '90% Sensitivity', fontweight = "bold", size = 12)
axs[0,1].legend(['Low Risk', 'High Risk'], loc='upper right')
plt.tight_layout()
fig.show()

In [ ]:
## Training Survival Curves

cutoff_levels = [
    '95% Sensitivity',
    '90% Sensitivity'
]

fig, axs = plt.subplots(3, 2, dpi = 300, figsize = (8,12), sharey = True, sharex = True)

i = 0

for cutoff_level in cutoff_levels:
    for model_name in model_names:
        cutoff = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results'][cutoff_level]['Threshold']
        print(cutoff_level, cutoff, model_name)
        
        ax = axs.ravel(order = 'F')[i]
        
        survival_results_data['training'][model_name][cutoff_level] = plotSurvivalFeat_Test(
            ax,
            df_train,
            df_test_ncdb,
            best_model_features[model_name]['features'],
            label = model_name_legend_label_dict[model_name],
            cutoff = cutoff
        )

        i = i + 1
        
        if math.isinf(survival_results_data['training'][model_name][cutoff_level]['OS']['HR']['Upper Bound']):
            ax.annotate("HR: N/A", (0, 0.938))
        else:
            ax.annotate(stringHR_from_data(survival_results_data['training'][model_name][cutoff_level]['OS']), (0, 0.938))

for ax, l in zip(axs.ravel(order = 'F'), ['A', 'B', 'C', 'D', 'E', 'F']):
    ax.get_legend().remove()
    ax.set_title(l, loc='left', fontweight='bold')
    ax.set_ylim([0.935, None])
    ax.yaxis.set_tick_params(labelleft = True)
    ax.set_xlabel('Months')
    ax.set_ylabel('Overall Survival')


plt.figtext(0.22, 1.0005, '95% Sensitivity', fontweight = "bold", size = 12)
plt.figtext(0.71, 1.0005, '90% Sensitivity', fontweight = "bold", size = 12)
axs[0,1].legend(['Low Risk', 'High Risk'], loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, dpi = 300, figsize = (10, 5), sharey = True)
i = 0
j = 0
for model_name in model_names[1:]:
    plotSurvivalFeatExternal_Path(
        ax[i],
        df_train,
        df_ucmc_parse_survival_val[(df_ucmc_parse_survival_val.regional_nodes_positive < 4)],
        best_model_features[model_name]['features'],
        label = model_name_legend_label_dict[model_name],
        cutoff = best_model_features[model_name]['subgroup_analyses']['training']['all']['senspec_results']['95% Sensitivity']['Threshold'],
        model = model
    )
    ax[i].set_ylim([0.86, 1.005])
    ax[i].legend()
    i = i + 1
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 2, dpi = 300, figsize = (8,10))

i = 0
for model_name in model_names:
    linearPlot(
        df_test_ncdb,
        df_train,
        LogisticRegression(max_iter = 1000, n_jobs = -1),
        use_weights = False,
        features = best_model_features[model_name]['features'],
        ax = axs.ravel(order = 'F')[i],
        title = model_name_legend_label_dict[model_name]
    )
    i = i + 1
    
for model_name in model_names:
    calibrationPlot(
        df_test_ncdb,
        df_train,
        LogisticRegression(max_iter = 1000, n_jobs = -1),
        use_weights = False,
        features = best_model_features[model_name]['features'],
        ax = axs.ravel(order = 'F')[i],
        title = model_name_legend_label_dict[model_name]
    )
    i = i + 1
    
for ax, l in zip(axs.ravel(order = 'F'), ['A', 'B', 'C', 'D', 'E', 'F']):
    ax.set_title(l, loc='left', fontweight='bold')
    
plt.tight_layout()
plt.show()

In [ ]:
res_sorted = dict(sorted(res.items(), key=lambda x:x[1]['llr'], reverse = True))

n = len(res)
width = 0.25
  
fig, ax1 = plt.subplots(dpi = 300, figsize = (10, 4.5))
l1 = ax1.bar(np.arange(n), [x[1]['llr'] for x in res_sorted.items()], color = 'b',
        width = width, edgecolor = 'black',
        )
ax1.set_xticks(np.arange(n) + width/2)
ax1.set_xticklabels(
    [pretty_feature_names_dict[x[0]] for x in res_sorted.items()],
    rotation = 90,
    ha = 'center'
)

ax2 = ax1.twinx()
l2 = ax2.bar(np.arange(n) + width, [x[1]['sample'] for x in res_sorted.items()], color = 'g',
        width = width, edgecolor = 'black',
        )

plt.legend([l1, l2], ["LLR", "N"])
plt.xlabel("Feature")
ax1.set_ylabel("Log-Likelihood Ratio (LLR)")
ax2.set_ylabel("Sample Size (N)")

plt.tight_layout()
  
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, dpi = 300, figsize = (7, 5), sharey = True)
axs = axs.ravel()

for i, model_name in enumerate(model_names):
    axs[i].plot(
        best_model_features[model_name]['results']['added_feature'], 
        best_model_features[model_name]['results']['AIC'] * 1e7,
        marker = 'o',
        markersize = 3
    )
    
    axs[i].set_xticklabels(
        [pretty_feature_names_dict[x] for x in best_model_features[model_name]['results']['added_feature']],
        rotation = 90
    )
    axs[i].set_title(model_name_legend_label_dict[model_name])
    axs[i].grid()
    

axs[0].set_ylabel('AIC ($\\times 10^7$)')

plt.tight_layout()
plt.show()

In [ ]:
survival_results_df = pd.DataFrame(
    columns = [
        'cohort',
        'model',
        'cutoff',
        'method',
        'n_patients',
        'features',
        'n_features',
        'HR',
        'HR_lowerbound',
        'HR_upperbound',
        'p', 
        'c-index'
    ]
)

for cohort in ['training', 'validation']:
    for model_name in model_names:
        for cutoff_level in cutoff_levels:
            results = survival_results_data[cohort][model_name][cutoff_level]
                                                                
            if cohort == 'training':
                survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': cutoff_level,
                    'method': "OS",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['OS']['HR']['HR'],
                    'HR_lowerbound': results['OS']['HR']['Lower Bound'],
                    'HR_upperbound': results['OS']['HR']['Upper Bound'],
                    'p': results['OS']['p'],
                    'c-index': results['OS']['c-index']
                }, ignore_index = True)
            elif cohort == 'validation':
                #RFI
                survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': cutoff_level,
                    'method': "RFI",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['Univariate']['HR']['HR'],
                    'HR_lowerbound': results['Univariate']['HR']['Lower Bound'],
                    'HR_upperbound': results['Univariate']['HR']['Upper Bound'],
                    'p': results['Univariate']['p'],
                    'c-index': results['Univariate']['c-index']
                }, ignore_index = True)
                
                #RFS
                survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': cutoff_level,
                    'method': "RFS",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['RFS']['HR']['HR'],
                    'HR_lowerbound': results['RFS']['HR']['Lower Bound'],
                    'HR_upperbound': results['RFS']['HR']['Upper Bound'],
                    'p': results['RFS']['p'],
                    'c-index': results['RFS']['c-index']
                }, ignore_index = True)
                
                #OS
                survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': cutoff_level,
                    'method': "OS",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['OS']['HR']['HR'],
                    'HR_lowerbound': results['OS']['HR']['Lower Bound'],
                    'HR_upperbound': results['OS']['HR']['Upper Bound'],
                    'p': results['OS']['p'],
                    'c-index': results['OS']['c-index']
                }, ignore_index = True)
                
for cohort in ['training', 'validation']:
    for model_name in model_names:   
        results = survival_results_data[cohort][model_name]['95% Sensitivity']
        
        if cohort == 'training':
            survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': 'Raw Model Prediction',
                    'method': "OS",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['OS_predict']['HR']['HR'],
                    'HR_lowerbound': results['OS_predict']['HR']['Lower Bound'],
                    'HR_upperbound': results['OS_predict']['HR']['Upper Bound'],
                    'p': results['OS_predict']['p'],
                    'c-index': results['OS_predict']['c-index']
                }, ignore_index = True)
        elif cohort == 'validation':
            ##RFI
            survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': 'Raw Model Prediction',
                    'method': "RFI",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['RFI_predict']['HR']['HR'],
                    'HR_lowerbound': results['RFI_predict']['HR']['Lower Bound'],
                    'HR_upperbound': results['RFI_predict']['HR']['Upper Bound'],
                    'p': results['RFI_predict']['p'],
                    'c-index': results['RFI_predict']['c-index']
                }, ignore_index = True)
            
            ##RFS
            survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': 'Raw Model Prediction',
                    'method': "RFS",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['RFS_predict']['HR']['HR'],
                    'HR_lowerbound': results['RFS_predict']['HR']['Lower Bound'],
                    'HR_upperbound': results['RFS_predict']['HR']['Upper Bound'],
                    'p': results['RFS_predict']['p'],
                    'c-index': results['RFS_predict']['c-index']
                }, ignore_index = True)
            
            ##OS
            survival_results_df = survival_results_df.append({
                    'model': model_name,
                    'cohort': cohort,
                    'cutoff': 'Raw Model Prediction',
                    'method': "OS",
                    'n_patients': results['Number of Patients'],
                    'features': results['Features'],
                    'n_features': len(results['Features']),
                    'HR': results['OS_predict']['HR']['HR'],
                    'HR_lowerbound': results['OS_predict']['HR']['Lower Bound'],
                    'HR_upperbound': results['OS_predict']['HR']['Upper Bound'],
                    'p': results['OS_predict']['p'],
                    'c-index': results['OS_predict']['c-index']
                }, ignore_index = True)

In [ ]:
subgroup_results_df = pd.DataFrame(
    columns = [
        'model',
        'subgroup_type',
        'subgroup_group',
        'n_patients',
        'features',
        'n_features',
        'cutoff',
        'thresh',
        'sens',
        'spec',
        'ppv',
        'npv',
        'AUC',
        'AUC_lowerbound',
        'AUC_upperbound',
        'AUPRC',
        'AUPRC_lowerbound',
        'AUPRC_upperbound'
    ]
)

for model_name in model_names:
    for subgroup_type in best_model_features[model_name]['subgroup_analyses'].keys():
        for subgroup_group in best_model_features[model_name]['subgroup_analyses'][subgroup_type].keys():
            print(model_name, subgroup_type, subgroup_group)
            results = best_model_features[model_name]['subgroup_analyses'][subgroup_type][subgroup_group]['senspec_results']
            
            if results != None:
                features = results['Features']
                n_patients = results['Number of Patients']
                n_features = len(features)
                auc = results['AUC']['AUC']
                auc_lb = results['AUC']['Lower Bound']
                auc_ub = results['AUC']['Upper Bound']
                auprc = results['AUPRC']['AUPRC']
                auprc_lb = results['AUPRC']['Lower Bound']
                auprc_ub = results['AUPRC']['Upper Bound']

                for cutoff_level in [
                    '95% Sensitivity',
                    '90% Sensitivity',
                    '95% Specificity',
                    '90% Specificity'
                ]:
                    subgroup_results_df = subgroup_results_df.append({
                        'model': model_name,
                        'subgroup_type': subgroup_type,
                        'subgroup_group': subgroup_group,
                        'cutoff': cutoff_level,
                        'n_patients': n_patients,
                        'features': features,
                        'n_features': n_features,
                        'thresh': results[cutoff_level]['Threshold'],
                        'sens': results[cutoff_level]['Sensitivity'],
                        'spec': results[cutoff_level]['Specificity'],
                        'ppv': results[cutoff_level]['PPV'],
                        'npv': results[cutoff_level]['NPV'],
                        'AUC': auc,
                        'AUC_lowerbound': auc_lb,
                        'AUC_upperbound': auc_ub,
                        'AUPRC': auprc,
                        'AUPRC_lowerbound': auprc_lb,
                        'AUPRC_upperbound': auprc_ub
                    }, ignore_index = True)
            else:
                subgroup_results_df = subgroup_results_df.append({
                    'model': model_name,
                    'subgroup_type': subgroup_type,
                    'subgroup_group': subgroup_group,
                    'cutoff': cutoff_level,
                    'n_patients': None,
                    'features': None,
                    'n_features': None,
                    'thresh': None,
                    'sens': None,
                    'spec': None,
                    'ppv': None,
                    'npv': None,
                    'AUC': None,
                    'AUC_lowerbound': None,
                    'AUC_upperbound': None,
                    'AUPRC': None,
                    'AUPRC_lowerbound': None,
                    'AUPRC_upperbound': None
                }, ignore_index = True)

## Tables

### Histologic Subgroup Analysis

In [ ]:
subgroup_histologic_aucs = (subgroup_results_df[
    (subgroup_results_df.subgroup_type == 'histologic') &
    (subgroup_results_df.cutoff == "95% Sensitivity")
].pivot(
    values = ['AUC', 'AUC_lowerbound', 'AUC_upperbound', 'AUPRC', 'AUPRC_lowerbound', 'AUPRC_upperbound'],
    columns = ['cutoff'],
    index = ['model', 'subgroup_group']
)
 .swaplevel(0, 1, axis = 1)
 .droplevel(level = 0, axis = 1)
 .sort_index(axis = 1)
 .applymap(lambda x: round(x, 2) if not pd.isna(x) else x)
 .astype(str)
)

subgroup_histologic_aucs['AUC (CI)'] = make_ci_text(subgroup_histologic_aucs, label = 'AUC')
subgroup_histologic_aucs['AUPRC (CI)'] = make_ci_text(subgroup_histologic_aucs, label = 'AUPRC')

subgroup_histologic_aucs.loc[['No Quant', 'ER/PR%', 'ER/PR%+Ki67%'], ['AUC (CI)', 'AUPRC (CI)']].rename(columns = {'AUC (CI)': 'AUROC (CI)'}).rename(model_name_legend_label_dict)

### Racial Subgroup Analysis

In [ ]:
subgroup_race_aucs = (subgroup_results_df[
    (subgroup_results_df.subgroup_type == 'race') &
    (subgroup_results_df.cutoff == "95% Sensitivity") &
    (subgroup_results_df.subgroup_group.isin(['Asian', 'Hispanic', 'Non-Hispanic Black', 'Non-Hispanic White']))
].pivot(
    values = ['AUC', 'AUC_lowerbound', 'AUC_upperbound', 'AUPRC', 'AUPRC_lowerbound', 'AUPRC_upperbound'],
    columns = ['cutoff'],
    index = ['model', 'subgroup_group']
)
 .swaplevel(0, 1, axis = 1)
 .droplevel(level = 0, axis = 1)
 .sort_index(axis = 1)
 .applymap(lambda x: round(x, 2) if not pd.isna(x) else x)
 .astype(str)
)

subgroup_race_aucs['AUC (CI)'] = make_ci_text(subgroup_race_aucs, label = 'AUC')
subgroup_race_aucs['AUPRC (CI)'] = make_ci_text(subgroup_race_aucs, label = 'AUPRC')

subgroup_race_aucs.loc[['No Quant', 'ER/PR%', 'ER/PR%+Ki67%'], ['AUC (CI)', 'AUPRC (CI)']].rename(columns = {'AUC (CI)': 'AUROC (CI)'}).rename(model_name_legend_label_dict)

### Nodal Subgroup Analysis

In [ ]:
subgroup_nodes_aucs = (subgroup_results_df[
    (subgroup_results_df.subgroup_type == 'nodes') &
    (subgroup_results_df.cutoff == "95% Sensitivity")
].pivot(
    values = ['AUC', 'AUC_lowerbound', 'AUC_upperbound', 'AUPRC', 'AUPRC_lowerbound', 'AUPRC_upperbound'],
    columns = ['cutoff'],
    index = ['model', 'subgroup_group']
)
 .swaplevel(0, 1, axis = 1)
 .droplevel(level = 0, axis = 1)
 .sort_index(axis = 1)
 .applymap(lambda x: round(x, 2) if not pd.isna(x) else x)
 .astype(str)
)

subgroup_nodes_aucs['AUC (CI)'] = make_ci_text(subgroup_nodes_aucs, label = 'AUC')
subgroup_nodes_aucs['AUPRC (CI)'] = make_ci_text(subgroup_nodes_aucs, label = 'AUPRC')

subgroup_nodes_aucs.loc[['No Quant', 'ER/PR%', 'ER/PR%+Ki67%'], ['AUC (CI)', 'AUPRC (CI)']].rename(columns = {'AUC (CI)': 'AUROC (CI)'}).rename(model_name_legend_label_dict)

### Overall Model Performance

In [ ]:
compare_aucs = (subgroup_results_df[
    ((subgroup_results_df.subgroup_group == 'test') |
    (subgroup_results_df.subgroup_group == 'external')) & 
    (subgroup_results_df.cutoff == '90% Sensitivity')
].pivot(
    values = ['AUC', 'AUC_lowerbound', 'AUC_upperbound', 'AUPRC', 'AUPRC_lowerbound', 'AUPRC_upperbound'],
    columns = ['subgroup_group'],
    index = ['model']
)
 .swaplevel(0, 1, axis = 1)
 .sort_index(axis = 1)
 .reindex(model_names)
 .applymap(lambda x: round(x, 2) if not pd.isna(x) else x)
 .astype(str)
)

In [ ]:
for cohort in ['test', 'external']:
    for m_i in ['AUC', 'AUPRC']:
        compare_aucs[(cohort, m_i + ' (CI)')] = compare_aucs[(cohort, m_i)] + " (" + compare_aucs[(cohort, m_i + '_lowerbound')] + " - " + compare_aucs[(cohort, m_i + '_upperbound')] + ")"

In [ ]:
(compare_aucs.swaplevel(0,1, axis = 1)[['AUC (CI)', 'AUPRC (CI)']]
 .swaplevel(0,1, axis = 1)
 .sort_index(axis = 1)
 .rename(columns = {'test': 'NCDB', 'external': 'UCMC', 'AUC (CI)': 'AUROC (CI)'})
 .rename(model_name_legend_label_dict)
)[['NCDB', 'UCMC']]

### Utility of Models as Rule-Out Tests

In [ ]:
(subgroup_results_df[
    ((subgroup_results_df.subgroup_group == 'test') |
    (subgroup_results_df.subgroup_group == 'external')) & 
    ((subgroup_results_df.cutoff == '90% Sensitivity') |
    (subgroup_results_df.cutoff == '95% Sensitivity'))
].pivot(
    values = ['sens', 'spec', 'ppv', 'npv'],
    columns = ['subgroup_group'],
    index = ['cutoff', 'model']
).swaplevel(0, 1, axis = 1)
 .sort_index(axis = 1)
 .reindex(model_names, level = 'model')
 .reindex(['95% Sensitivity', '90% Sensitivity'], level = 'cutoff')
 .applymap(lambda x: round(x, 2) if not pd.isna(x) else x)
 .rename(columns={'test': 'NCDB', 'external': 'UCMC', 'npv': 'NPV', 'ppv': 'PPV', 'sens': 'Sen', 'spec': 'Spe'})
 .rename(model_name_legend_label_dict)
 .reindex(['Sen', 'Spe', 'PPV', 'NPV'], axis=1, level=1)
)

### Survival Analysis in UCMC Cohort

In [ ]:
survival_results_df_pivot = (survival_results_df.pivot(
    values = ['HR', 'HR_lowerbound', 'HR_upperbound', 'p', 'c-index'],
    columns = ['cohort', 'method'],
    index = ['cutoff', 'model']
)
 .swaplevel(0, 1, axis = 1).swaplevel(1, 2, axis = 1).sort_index(axis = 1)
 .applymap(lambda x: round(x, 2) if not pd.isna(x) else x)
 .astype(str)
)

survival_results_df_pivot[('training', 'OS', 'HR (CI)')] = survival_results_df_pivot[('training', 'OS', 'HR')] + " (" + survival_results_df_pivot[('training', 'OS', 'HR_lowerbound')] + " - " + survival_results_df_pivot[('training', 'OS', 'HR_upperbound')] + ")"

for m_i in ['OS', 'RFI', 'RFS']:
    survival_results_df_pivot[('validation', m_i, 'HR (CI)')] = survival_results_df_pivot[('validation', m_i, 'HR')] + " (" + survival_results_df_pivot[('validation', m_i, 'HR_lowerbound')] + " - " + survival_results_df_pivot[('validation', m_i, 'HR_upperbound')] + ")"

(survival_results_df_pivot.swaplevel(0,2, axis = 1)[['HR (CI)', 'p', 'c-index']]
 .swaplevel(0,2, axis = 1)
 .sort_index(axis = 1)
 .reindex(model_names, level = 1)
 .reindex(['RFI', 'RFS', 'OS'], level = 1, axis = 1).
 rename(model_name_legend_label_dict)
)['validation']

### Survival Analysis in NCDB Cohort

In [ ]:
(survival_results_df_pivot.swaplevel(0,2, axis = 1)[['HR (CI)', 'p', 'c-index']]
 .swaplevel(0,2, axis = 1)
 .sort_index(axis = 1)
 .reindex(model_names, level = 1)
 .reindex(['RFI', 'RFS', 'OS'], level = 1, axis = 1)
 .rename(model_name_legend_label_dict)
)['training']['OS']

### Z-scores and p-values for comparison of AUROC in validation cohort

In [ ]:
auc_compare_pval

In [ ]:
auc_compare_z

### Median follow-up time in test and validation cohort

In [ ]:
df_ucmc_parse_survival_val['year_FU'].median()*12

# Pickles for Calculator

In [ ]:
pickle_dir = '/home/asim/NCDB_Projects/NCDBRS-main/pickle/'

for model_name in model_names:
    model = LogisticRegression(max_iter = 1000, n_jobs = -1)
    df_subset_train = df_train[best_model_features[model_name]['features'] + ['high_odx']]
    model.fit(df_subset_train[best_model_features[model_name]['features']], df_subset_train['high_odx'])
    
    with open(pickle_dir + model_name_legend_file_dict[model_name] + '.pickle', 'wb') as f:
        pickle.dump(model, f)

In [ ]:
with open(pickle_dir + 'model_features.pickle', 'wb') as f:
    pickle.dump(best_model_features, f)

In [ ]:
print(best_model_features['No Quant']['subgroup_analyses']['all'])

In [ ]:
feat_dict = {}

for model_name in model_names:
    feat_dict[model_name_legend_file_dict[model_name]] = best_model_features[model_name]['features']

In [ ]:
AUROCFeatExternal_pickle(
    df_train, df_ucmc_parse_odx_val, feat_dict,
    model = model,
    pickle_dir = pickle_dir
)

In [ ]:
AUROCFeat_pickle(
    pd.concat([df_train, df_test_ncdb]), feat_dict,
    model = model,
    pickle_dir = pickle_dir
)

In [ ]:
export_survival_pickle(
    df_train, df_ucmc_parse_survival_val, feat_dict,
    model = model,
    pickle_dir = pickle_dir
)

# Gridsearch Results

In [ ]:
gridsearch_dir = '/home/asim/NCDB_Projects/test/gridsearch/'

In [ ]:
gridsearches = {
    'Logistic Regression': gridsearch_fromfile(gridsearch_dir + 'logistic_regression.txt'),
    'Random Forest': gridsearch_fromfile(gridsearch_dir + 'random_forest.txt'),
    'Adaboost': gridsearch_fromfile(gridsearch_dir + 'adaboost.txt'),
    'Neural Network: 1 Layer': gridsearch_fromfile(gridsearch_dir + 'neuralnetwork1.txt'),
    'Neural Network: 2 Layers': gridsearch_fromfile(gridsearch_dir + 'neuralnetwork2.txt')
}

In [ ]:
for k,v in gridsearches.items():
    print(k.upper())
    print('Best Parameters:')
    print(v.loc[v['AUC'].idxmax(), 'Parameters'])
    print('AUC: %.3f, AIC: %.3e' % (v.loc[v['AUC'].idxmax(), 'AUC'], v.loc[v['AUC'].idxmax(), 'AIC']))
    print('Features: %s' % str(v.loc[v['AUC'].idxmax(), 'Features']))
    print()